# Llama Index, NBA Knowledge-base Vector Store

Create a Vector Store with Llama index and CassIO, with NBA data to use as a search engine and text generator, backed by Astra DB. Demonstrate multiple Llama index modules and question answering using the framework, including a Gradio Web Interface.

Note: much of dataset build code is credited to lifeofbaka (https://github.com/lifeofbaka)

## Colab-specific setup

Make sure you have a Database and get ready to upload the Secure Connect Bundle and supply the Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details. Remember you need a custom Token with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).

_Note: this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/llamaindex/vector-quickstart/)._


In [4]:
# install required dependencies
! pip install -q --progress-bar off \
    "cassio>=0.1.0" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "llama-index==0.8.8" \
    "nltk==3.8.1" \
    "pypdf==3.15.2" \
    "python-dotenv==1.0.0" \
    "openai" \
    "langchain" \
    "pypdf" \
    "PyCryptodome" \
    "gradio" \
    "wikipedia" \
    "bs4" \
    "transformers"
exit()

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


⚠️ **Do not mind a "Your session crashed..." message you may see.**

It was us, making sure your kernel restarts with all the correct dependency versions. _You can now proceed with the notebook._

In [1]:
# imports
from bs4 import BeautifulSoup
import numpy as np
import requests
import re
import pandas as pd
import io
import csv
import wikipedia
from typing import Any, List
import json
import os
import re
from urllib.request import urlopen
import openai
from pathlib import Path
from llama_index import download_loader
from llama_index import (
    GPTVectorStoreIndex,
    GPTSimpleKeywordTableIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext
)
from llama_index import StorageContext, ServiceContext, GPTVectorStoreIndex, LLMPredictor, PromptHelper, SimpleDirectoryReader, load_index_from_storage
from langchain.llms.openai import OpenAIChat
from langchain.chat_models import ChatOpenAI

In [2]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name ')

Your Astra DB Keyspace namevector_preview


In [35]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
from getpass import getpass
ASTRA_DB_APPLICATION_TOKEN = getpass('Your Astra DB Token ("AstraCS:..."): ')

Your Astra DB Token ("AstraCS:..."): ··········


### Astra DB Secure Connect Bundle

Please upload the Secure Connect Bundle zipfile to connect to your Astra DB instance.

The Secure Connect Bundle is needed to establish a secure connection to the database.
Click [here](https://awesome-astra.github.io/docs/pages/astra/download-scb/#c-procedure) for instructions on how to download it from Astra DB.

In [36]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

Please upload your Secure Connect Bundle


Saving secure-connect-vectordemo.zip to secure-connect-vectordemo.zip


In [37]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                "token",
                ASTRA_DB_APPLICATION_TOKEN,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unsupported CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unsupported CQL Session mode')

### LLM Provider

In the cell below you can choose between **GCP Vertex AI** or **OpenAI** for your LLM services.
(See [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for more details).

Make sure you set the `llmProvider` variable and supply the corresponding access secrets in the following cell.

In [38]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI', 'Azure_OpenAI'


In [39]:
from getpass import getpass
if llmProvider == 'OpenAI':
    apiSecret = getpass(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
elif llmProvider == 'Azure_OpenAI':
    # a few parameters must be input
    apiSecret = input(f'Your API Key for LLM provider "{llmProvider}": ')
    os.environ['AZURE_OPENAI_API_KEY'] = apiSecret
    apiBase = input('The "Base URL" for your models (e.g. "https://YOUR-RESOURCE-NAME.openai.azure.com"): ')
    os.environ['AZURE_OPENAI_API_BASE'] = apiBase
    apiLLMDepl = input('The name of your LLM Deployment: ')
    os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'] = apiLLMDepl
    apiLLMModel = input('The name of your LLM Model (e.g. "gpt-4"): ')
    os.environ['AZURE_OPENAI_LLM_MODEL'] = apiLLMModel
    apiEmbDepl = input('The name for your Embeddings Deployment: ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'] = apiEmbDepl
    apiEmbModel = input('The name of your Embedding Model (e.g. "text-embedding-ada-002"): ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'] = apiEmbModel

    # The following is probably not going to change for some time...
    os.environ['AZURE_OPENAI_API_VERSION'] = '2023-03-15-preview'
else:
    raise ValueError('Unknown/unsupported LLM Provider')

Your secret for LLM provider "OpenAI": ··········


### Wikipedia Dataset
using help from lifeofbacka to retrieve a dataset from wikipedia. We will focus on NBA teams as the topic of interest. And for demo purpose cut the dataset down even more, to one team.

In [8]:
#wiki links
wiki_txt_links = ['https://raw.githubusercontent.com/lifeofbaka/Datasets/main/Wikipedia/wiki' + str(i)+ '.txt' for i in range(1,9)] #if wiki size changes update the range.

In [ ]:
# topics

# basketball_teams = [
#     "Atlanta Hawks",
#     "Boston Celtics",
#     "Brooklyn Nets",
#     "Charlotte Hornets",
#     "Chicago Bulls",
#     "Cleveland Cavaliers",
#     "Dallas Mavericks",
#     "Denver Nuggets",
#     "Detroit Pistons",
#     "Golden State Warriors",
#     "Houston Rockets",
#     "Indiana Pacers",
#     "Los Angeles Clippers",
#     "Los Angeles Lakers",
#     "Memphis Grizzlies",
#     "Miami Heat",
#     "Milwaukee Bucks",
#     "Minnesota Timberwolves",
#     "New Orleans Pelicans",
#     "New York Knicks",
#     "Oklahoma City Thunder",
#     "Orlando Magic",
#     "Philadelphia 76ers",
#     "Phoenix Suns",
#     "Portland Trail Blazers",
#     "Sacramento Kings",
#     "San Antonio Spurs",
#     "Toronto Raptors",
#     "Utah Jazz",
#     "Washington Wizards"
# ]
'''

In [5]:
# smaller test case with a single team
basketball_teams = [ "Utah Jazz"]

In [6]:
len(basketball_teams)

1

In [9]:
# Wiki Extraction
# find ever wiki that has one of the values in the topic case. Then add values to a list to be read later

nba_wiki = []
for i in range(len(wiki_txt_links)):
   data = urlopen(wiki_txt_links[i]) # it's a file like object and works just like a file
   articles = [line.strip() for line in data] # files are iterable
   for team in basketball_teams:
      for topic in articles:
        if team in str(topic):
          pattern = r'\xe2\x80\x93'
          matches = re.findall(pattern, str(topic))
          if not matches:
            nba_wiki.append(str(topic)[2:-1])
print (len(nba_wiki))

181


In [10]:
nba_wiki

['1978-79 Utah Jazz season',
 '1978\\xe2\\x80\\x9379 Utah Jazz season',
 '1979-80 Utah Jazz',
 '1979-80 Utah Jazz season',
 '1979\\xe2\\x80\\x9380 Utah Jazz',
 '1979\\xe2\\x80\\x9380 Utah Jazz season',
 '1980-81 Utah Jazz',
 '1980-81 Utah Jazz season',
 '1980\\xe2\\x80\\x9381 Utah Jazz',
 '1980\\xe2\\x80\\x9381 Utah Jazz season',
 '1981-82 Utah Jazz',
 '1981-82 Utah Jazz season',
 '1981\\xe2\\x80\\x9382 Utah Jazz',
 '1981\\xe2\\x80\\x9382 Utah Jazz season',
 '1982-83 Utah Jazz',
 '1982-83 Utah Jazz season',
 '1982\\xe2\\x80\\x9383 Utah Jazz',
 '1982\\xe2\\x80\\x9383 Utah Jazz season',
 '1983-84 Utah Jazz',
 '1983-84 Utah Jazz season',
 '1983\\xe2\\x80\\x9384 Utah Jazz',
 '1983\\xe2\\x80\\x9384 Utah Jazz season',
 '1984-85 Utah Jazz',
 '1984-85 Utah Jazz season',
 '1984\\xe2\\x80\\x9385 Utah Jazz',
 '1984\\xe2\\x80\\x9385 Utah Jazz season',
 '1985-86 Utah Jazz',
 '1985-86 Utah Jazz season',
 '1985\\xe2\\x80\\x9386 Utah Jazz',
 '1985\\xe2\\x80\\x9386 Utah Jazz season',
 '1986-87 Utah Jaz

In [11]:
nba_wiki_ = set()
for i in range(len(nba_wiki)):
  try:
    if nba_wiki[i] in nba_wiki[i+1]:
      nba_wiki_.add(nba_wiki[i+1])
    else:
      nba_wiki_.add(nba_wiki(i))
  except:
    nba_wiki_.add(nba_wiki[i])
nba_wiki_ = list(nba_wiki_)
print (len(nba_wiki_))

95


In [12]:
sorted(nba_wiki_)

['1978-79 Utah Jazz season',
 '1978\\xe2\\x80\\x9379 Utah Jazz season',
 '1979-80 Utah Jazz season',
 '1979\\xe2\\x80\\x9380 Utah Jazz season',
 '1980-81 Utah Jazz season',
 '1980\\xe2\\x80\\x9381 Utah Jazz season',
 '1981-82 Utah Jazz season',
 '1981\\xe2\\x80\\x9382 Utah Jazz season',
 '1982-83 Utah Jazz season',
 '1982\\xe2\\x80\\x9383 Utah Jazz season',
 '1983-84 Utah Jazz season',
 '1983\\xe2\\x80\\x9384 Utah Jazz season',
 '1984-85 Utah Jazz season',
 '1984\\xe2\\x80\\x9385 Utah Jazz season',
 '1985-86 Utah Jazz season',
 '1985\\xe2\\x80\\x9386 Utah Jazz season',
 '1986-87 Utah Jazz season',
 '1986\\xe2\\x80\\x9387 Utah Jazz season',
 '1987-88 Utah Jazz season',
 '1987\\xe2\\x80\\x9388 Utah Jazz season',
 '1988-89 Utah Jazz season',
 '1988\\xe2\\x80\\x9389 Utah Jazz season',
 '1989-90 Utah Jazz season',
 '1989\\xe2\\x80\\x9390 Utah Jazz season',
 '1990-91 Utah Jazz season',
 '1990\\xe2\\x80\\x9391 Utah Jazz season',
 '1991-92 Utah Jazz season',
 '1991\\xe2\\x80\\x9392 Utah Jazz s

In [14]:
# remove duplicates
to_remove = ['(football)','(NHL)', '(AFL)', '(hockey)', '(AFL, 1926)', '(American football)', '(WFL)', '(UHL)']
for line in nba_wiki_:
  for item in to_remove:
    if item in line:
      print (line)
      nba_wiki_.remove(line)
print (len(nba_wiki_))


95


In [15]:
#load data function. The function is reliant on a number of table finder functions that have also been assigned below as draft picks and roster.
# The functions find specific table values within the pagent context using Beautiful Soup then output a dictionary from a working dataframe which is then stored within the results dictionary and usable sub topic storage for JSON.

#test
#articles = ['1968–69 Atlanta Hawks season','1968–69_Chicago_Bulls_season']


#WIKIPEDIA PULL
def load_data( pages: List[str], lang: str = "en", **load_kwargs: Any):
    results = []
    for page in pages:
        wikipedia.set_lang(lang)
        try:
          page_content = wikipedia.page(page, **load_kwargs).content
          summary = page_content
          matches = re.findall(r'==\s(.*?)\s==', summary)

          if 'Draft picks' in matches:
              draftpick = draft_picks(page)
          else:
              draftpick = ''
          if 'Roster' in matches:
              team_roster = roster(page)
              pull.append(page)
          else:
              team_roster = ['','']

          data = {page: {"summary": summary, "Draft Pick": draftpick, "Coach": team_roster[0], "Roster": team_roster[1]}}
          results.append(data)
        except:
          pass

    return results

#Tables = re.findall(r"\n== (.*?) ==\n", summary[0])
#print (Tables)


#################################################################################################


def draft_picks(article):
    url = requests.get(f'https://en.wikipedia.org/wiki/{article}').content
    soup = BeautifulSoup(url, 'html.parser')
    tables = []
    for i in soup.find_all('table'):
        tables.append(i)
    for i in range(len(tables)):
        try:
          table_text = tables[i].text
          table_text = table_text.split('\n')

          # find table columns
          col_finder = ' '.join(table_text)
          cols = col_finder.split('   ')
          cols = cols[0].split(' ')

          result_cols = []
          for item in cols:
              if item != '':
                  result_cols.append(item)


          #find table data
          result = []
          pattern = r"(?<=\xa0)\w++"
          for record in table_text:
              match = re.search(r"\xa0(.*)", record)
              if match:
                  result.append(match.group(1))
              else:
                  if record != '':
                      result.append(record)
          result = result[len(result_cols):]
          result = np.array(result)
          shaper = (len(result_cols),int(result.shape[0]/len(result_cols)))

          result = result.reshape(shaper[1],shaper[0])

          df = pd.DataFrame(result, columns=result_cols)
          dictionary = df.to_dict()
          return dictionary
        except:
            pass
    return "could not find table"


def roster(article):
    url = requests.get(f'https://en.wikipedia.org/wiki/{article}').content
    soup = BeautifulSoup(url, 'html.parser')
    tables = []
    for i in soup.find_all('table'):
        tables.append(i)

    for i in range(len(tables)):
        try:
            table_text = tables[i].text
            table_text = table_text.split('\n')

            # find table columns
            col_finder = ' '.join(table_text)
            cols = col_finder.split('   ')
            cols = ' '.join(cols[1:4]).split(' ')

            result_cols = []
            for i in cols:
                if i != '':
                    result_cols.append(i)
            result_cols = result_cols[2:7] + [' '.join(result_cols[7:9])] + result_cols[-1:]


            #find table data
            result = []
            for record in table_text:
                matches = re.findall(r'\xa0', record)
                if matches:
                    result.append(record.replace('\xa0',' '))
                else:
                    if record != '':
                        result.append(record)
            result = result[result.index('From') + 1:result.index('Head coach')]


            #Head Coach


            try:

                shaper = (len(result_cols), int(len(result) / len(result_cols)))
                n_result = np.array(result)
                result = n_result.reshape(shaper[1], shaper[0])

                df = pd.DataFrame(result, columns=result_cols)
                roster_dictionary = df.to_dict()
                coach = {table_text[table_text.index('Head coach'): table_text.index('Head coach') + 1][0]:
                             table_text[table_text.index('Head coach') + 1: table_text.index('Head coach') + 2][0]}

                return coach, roster_dictionary
            except:
                result_cols.remove('DOB (YYYY-MM-DD)')
                result = np.array(result)
                shaper = (len(result_cols), int(result.shape[0] / len(result_cols)))
                result = result.reshape(shaper[1], shaper[0])

                df = pd.DataFrame(result, columns=result_cols)
                new_column = [None for i in range(len(df))]
                position = df.columns.get_loc('Weight') + 1
                df.insert(position, 'DOB (YYYY-MM-DD)', new_column)
                roster_dictionary = df.to_dict()
                coach = {table_text[table_text.index('Head coach'): table_text.index('Head coach') + 1][0]:
                             table_text[table_text.index('Head coach') + 2: table_text.index('Head coach') + 3][0]}

                return coach, roster_dictionary
        except:
            pass
    return "could not find table"

In [16]:
test_i = nba_wiki_.index("1996-97 Utah Jazz season")
test_i

34

In [17]:
nba_wiki_[34]

'1996-97 Utah Jazz season'

In [18]:
draft_picks([nba_wiki_[34]])

'could not find table'

### JSON

Call the load function for every value previously extraced and write a JSON file

In [19]:
len(nba_wiki_)

95

In [20]:
pull = []
for value in load_data(pages=sorted(nba_wiki_)):
    for key in value:
      file_name = key.replace("-"," ")
      file_name = file_name.replace(" ","_")
      pull.append(file_name)
      with open(f"{file_name}.json", "w") as file:
        json.dump(value, file, indent=4)

<ipython-input-15-e2cf4fec0076>:81: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dictionary = df.to_dict()
<ipython-input-15-e2cf4fec0076>:81: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dictionary = df.to_dict()
<ipython-input-15-e2cf4fec0076>:81: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dictionary = df.to_dict()
<ipython-input-15-e2cf4fec0076>:81: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dictionary = df.to_dict()


In [21]:
#files in content
sorted(pull)

['1978_79_Utah_Jazz_season',
 '1979-80 Utah Jazz season',
 '1979_80_Utah_Jazz_season',
 '1980-81 Utah Jazz season',
 '1980_81_Utah_Jazz_season',
 '1981-82 Utah Jazz season',
 '1981_82_Utah_Jazz_season',
 '1982-83 Utah Jazz season',
 '1982_83_Utah_Jazz_season',
 '1983-84 Utah Jazz season',
 '1983_84_Utah_Jazz_season',
 '1984-85 Utah Jazz season',
 '1984_85_Utah_Jazz_season',
 '1985-86 Utah Jazz season',
 '1985_86_Utah_Jazz_season',
 '1986-87 Utah Jazz season',
 '1986_87_Utah_Jazz_season',
 '1987-88 Utah Jazz season',
 '1987_88_Utah_Jazz_season',
 '1988-89 Utah Jazz season',
 '1988_89_Utah_Jazz_season',
 '1989-90 Utah Jazz season',
 '1989_90_Utah_Jazz_season',
 '1990-91 Utah Jazz season',
 '1990_91_Utah_Jazz_season',
 '1991-92 Utah Jazz season',
 '1991_92_Utah_Jazz_season',
 '1992-93 Utah Jazz season',
 '1992_93_Utah_Jazz_season',
 '1993-94 Utah Jazz season',
 '1993_94_Utah_Jazz_season',
 '1994-95 Utah Jazz season',
 '1994_95_Utah_Jazz_season',
 '1995-96 Utah Jazz season',
 '1995_96_Utah

In [22]:
print (len(pull))
print(len(nba_wiki_))

94
95


### Load JSON files into Llama Index

single document test case

In [23]:
print (os.listdir())
print (len(os.listdir()))

['.config', '2006_07_Utah_Jazz_season.json', '1979_80_Utah_Jazz_season.json', '2014_15_Utah_Jazz_season.json', '1980_81_Utah_Jazz_season.json', '1999_2000_Utah_Jazz_season.json', '1986_87_Utah_Jazz_season.json', '2008_09_Utah_Jazz_season.json', '2019_20_Utah_Jazz_season.json', '1984_85_Utah_Jazz_season.json', '2012_13_Utah_Jazz_season.json', '1987_88_Utah_Jazz_season.json', '2018_19_Utah_Jazz_season.json', '2001_02_Utah_Jazz_season.json', '2021_22_Utah_Jazz_season.json', '2000_01_Utah_Jazz_season.json', '2017_18_Utah_Jazz_season.json', '1994_95_Utah_Jazz_season.json', '1982_83_Utah_Jazz_season.json', '2023_24_Utah_Jazz_season.json', '1988_89_Utah_Jazz_season.json', '2002_03_Utah_Jazz_season.json', '1989_90_Utah_Jazz_season.json', '2005_06_Utah_Jazz_season.json', '1996_97_Utah_Jazz_season.json', '1999_00_Utah_Jazz_season.json', '1995_96_Utah_Jazz_season.json', '2022_23_Utah_Jazz_season.json', '1990_91_Utah_Jazz_season.json', '2003_04_Utah_Jazz_season.json', '2016_17_Utah_Jazz_season.jso

In [24]:
p =os.listdir()[1:-1]

In [25]:
file_path = "/content/"

In [26]:
pull_ = set(i for i in pull)
pull

['1979-80 Utah Jazz season',
 '1980-81 Utah Jazz season',
 '1981-82 Utah Jazz season',
 '1982-83 Utah Jazz season',
 '1983-84 Utah Jazz season',
 '1984-85 Utah Jazz season',
 '1985-86 Utah Jazz season',
 '1986-87 Utah Jazz season',
 '1987-88 Utah Jazz season',
 '1988-89 Utah Jazz season',
 '1989-90 Utah Jazz season',
 '1990-91 Utah Jazz season',
 '1991-92 Utah Jazz season',
 '1992-93 Utah Jazz season',
 '1993-94 Utah Jazz season',
 '1994-95 Utah Jazz season',
 '1995-96 Utah Jazz season',
 '1996-97 Utah Jazz season',
 '1997-98 Utah Jazz season',
 '1998-99 Utah Jazz season',
 '1999-00 Utah Jazz season',
 '1999-2000 Utah Jazz season',
 '2000-01 Utah Jazz season',
 '2001-02 Utah Jazz season',
 '2002-03 Utah Jazz season',
 '2003-04 Utah Jazz season',
 '2004-05 Utah Jazz season',
 '2005-06 Utah Jazz season',
 '2006-07 Utah Jazz season',
 '2007-08 Utah Jazz season',
 '2008-09 Utah Jazz season',
 '2009-10 Utah Jazz season',
 '2010-11 Utah Jazz season',
 '2011-12 Utah Jazz season',
 '2012-13 Ut

In [27]:
paths = []
for i in pull:
  i = i.replace("-", " ")
  i = i.replace(" ",("_"))
  paths.append(file_path +i+".json")

In [28]:
print(paths[0:2])

['/content/1979_80_Utah_Jazz_season.json', '/content/1980_81_Utah_Jazz_season.json']


In [29]:
from pathlib import Path
from llama_index import download_loader

JSONReader = download_loader("JSONReader")

loader = JSONReader()


In [30]:
paths[0]

'/content/1979_80_Utah_Jazz_season.json'

In [31]:
document = loader.load_data(Path(paths[34]))

In [32]:
#example of what the fully working JSON file should look like
document

[Document(id_='4898a81a-f9b3-4f10-a0cf-eefcfb835256', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='9e0f2005d6d0055513f2afaa90a54f1d67af02ad818cd33b2313a36dd8fd3c86', text='"2012-13 Utah Jazz season"', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')]

In [34]:
print('Document ID:', document[0].doc_id, 'Document Hash:', document[0].hash)

Document ID: 4898a81a-f9b3-4f10-a0cf-eefcfb835256 Document Hash: 9e0f2005d6d0055513f2afaa90a54f1d67af02ad818cd33b2313a36dd8fd3c86


### Colab preamble completed

## The following cells constitute the demo notebook proper.

# LlamaIndex, Vector Store

Create a Vector Store with LamaIndex and CassIO, and build a powerful search engine and text generator, backed by Astra DB / Apache Cassandra®.

### Table of contents:

- Setup (database, LLM+embeddings)
- Create Astra vector store
- Insert documents in it
- Test answering questions using a single doc with AStra Vector Search, CassIO and LlamaIndex
- Use Gradio web interface to showcase Q&A across multiple docs
- Cleanup (optional)

_**NOTE:** this uses Cassandra's "Vector Similarity Search" capability.
Make sure you are connecting to a vector-enabled database for this demo._

## Setup

In [40]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext

This is the LlamaIndex class providing support for Astra DB / Cassandra:

In [41]:
from llama_index.vector_stores import CassandraVectorStore

A database connection is needed to access Cassandra. The following assumes
that a _vector-search-capable Astra DB instance_ is available. Adjust as needed.

In [42]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(134505198303840) a4c29bc6-b900-44c5-b980-584a0c4542b5-us-east1.db.astra.datastax.com:29042:7c9fc34c-2841-4389-b862-25badb95e6a3> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Both an LLM and an embedding function are required.

Below is the logic to instantiate the LLM and embeddings of choice. We chose to leave it in the notebooks for clarity.

In [43]:
import os
# creation of the LLM resources


if llmProvider == 'OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'open_ai'
    from llama_index.llms import OpenAI
    from llama_index.embeddings import OpenAIEmbedding
    llm = OpenAI(temperature=0)
    myEmbedding = OpenAIEmbedding()
    vector_dimension = 1536
    print("LLM+embeddings from OpenAI")
elif llmProvider == 'GCP_VertexAI':
    from llama_index import LangchainEmbedding
    # LlamaIndex lets you plug any LangChain's LLM+embeddings:
    from langchain.llms import VertexAI
    from langchain.embeddings import VertexAIEmbeddings
    llm = VertexAI()
    lcEmbedding = VertexAIEmbeddings()
    vector_dimension = len(lcEmbedding.embed_query("This is a sample sentence."))
    # ... if you take care of wrapping the LangChain embedding like this:
    myEmbedding = LangchainEmbedding(lcEmbedding)
    print("LLM+embeddings from Vertex AI")
elif llmProvider == 'Azure_OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'azure'
    os.environ['OPENAI_API_VERSION'] = os.environ['AZURE_OPENAI_API_VERSION']
    os.environ['OPENAI_API_BASE'] = os.environ['AZURE_OPENAI_API_BASE']
    os.environ['OPENAI_API_KEY'] = os.environ['AZURE_OPENAI_API_KEY']
    from llama_index import LangchainEmbedding
    # LlamaIndex lets you plug any LangChain's LLM+embeddings:
    from langchain.llms import AzureOpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = AzureOpenAI(temperature=0, model_name=os.environ['AZURE_OPENAI_LLM_MODEL'],
                      engine=os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'])
    lcEmbedding = OpenAIEmbeddings(model=os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'],
                                   deployment=os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'])
    vector_dimension = len(lcEmbedding.embed_query("This is a sample sentence."))
    # ... if you take care of wrapping the LangChain embedding like this:
    myEmbedding = LangchainEmbedding(lcEmbedding)
    print('LLM+embeddings from Azure OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

print(f"Vector dimension for this embedding model: {vector_dimension}")

LLM+embeddings from OpenAI
Vector dimension for this embedding model: 1536


The following cell ensures that throughout all of LlamaIndex the LLM and embedding function instantiated above will be used:

In [44]:
from llama_index import ServiceContext
from llama_index import set_global_service_context

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=myEmbedding,
    chunk_size=256,
)
set_global_service_context(service_context)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Create vector store

In [49]:
table_name = 'vs_nba_ll_' + llmProvider

In the LlamaIndex abstractions, the `CassandraVectorStore` instance is best wrapped into the creation of a "storage context", which you'll momentarily use to create the index proper:

In [50]:
storage_context = StorageContext.from_defaults(
    vector_store = CassandraVectorStore(
        session=session,
        keyspace=keyspace,
        table=table_name,
        embedding_dimension=vector_dimension,
        insertion_batch_size=15,
    )
)

## Insert documents into Astra

In [47]:
titles = pull

In [48]:
print(titles[0])
print(len(titles))

1979-80 Utah Jazz season
94


In [52]:
from llama_index.storage.storage_context import StorageContext

openai.api_key = apiSecret

#if index_name not in pinecone.list_indexes():
 # pinecone.create_index(name=index_name, dimension=1536, metric="cosine")
  # pinecone.create_index(index_name, dimension=len(embeds[0]))
# connect to index
# index = pinecone.Index(index_name)
# pinecone.create_index(name=index_name, dimension=1536, metric="cosine")

aa_indices = {}
index_summaries = {}

for astra_title, name in zip(titles, paths):
    metadata_filters = {"name": name}
    storage_context = StorageContext.from_defaults(
      vector_store = CassandraVectorStore(
        session=session,
        keyspace=keyspace,
        table=table_name,
        embedding_dimension=vector_dimension,
        insertion_batch_size=15,
      )
    )

    #aa_docs = SimpleDirectoryReader(input_files=[i for i in paths]).load_data()


    #try:
    document = loader.load_data(Path(name))
    aa_indices[name] = GPTVectorStoreIndex.from_documents(document, storage_context=storage_context)
    aa_indices[name].index_struct.index_id = astra_title
    #except:
    #  pass

    index_summaries[name] = f"Wikipedia article about {name}"

In [53]:
from llama_index.indices.composability.graph import ComposableGraph

graph = ComposableGraph.from_indices(
    GPTSimpleKeywordTableIndex,
    [index for _, index in aa_indices.items()],
    [summary for _, summary in index_summaries.items()],
    max_keywords_per_chunk=50
)

llm_predictor_chatgpt = LLMPredictor(
    #llm=OpenAIChat(temperature=0, model_name="gpt-4")
    llm=OpenAIChat(temperature=0, model_name="gpt-3.5-turbo")
)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor_chatgpt)

custom_query_engines = {
    graph.root_id: graph.root_index.as_query_engine(retriever_mode='simple', service_context=service_context)
}

query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)

query_str = "Tell me more about the Utah Jazz"
response_chatgpt = query_engine.query(query_str)

print(response_chatgpt)
print(response_chatgpt.get_formatted_sources())

None



[nltk_data] Downloading package stopwords to /tmp/llama_index...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## Answer questions

In [63]:
query_str = "Tell me more about the 1996_97_Utah_Jazz_season"
response_chatgpt = query_engine.query(query_str)

if not response_chatgpt or not response_chatgpt.get_formatted_sources():
    print("I am sorry, but your query is outside the boundaries of what I was trained on.")
else:
    print(response_chatgpt)
    print(response_chatgpt.get_formatted_sources())

The 1996-97 Utah Jazz season was a significant season for the team.
> Source (Doc id: 4a44e8af-eec4-46ff-b894-484d7c5ec146): The 1996-97 Utah Jazz season was a significant season for the team....

> Source (Doc id: 17aebcbb-0093-4e06-b00b-1220d8eff22a): "1996-97 Utah Jazz season"...

> Source (Doc id: 76a6762f-4337-4a77-9b46-beb08a4080eb): "1996-97 Utah Jazz season"...


In [61]:
query_str = "Tell me more about the 1996_97_Utah_Jazz_season Roster"
response_chatgpt = query_engine.query(query_str)

if not response_chatgpt or not response_chatgpt.get_formatted_sources():
    print("I am sorry, but your query is outside the boundaries of what I was trained on.")
else:
    print(response_chatgpt)
    print(response_chatgpt.get_formatted_sources())

The 1996-97 Utah Jazz season roster was a group of players who played for the Utah Jazz during that specific season. Unfortunately, without further information, it is not possible to provide specific details about the players on the roster.
> Source (Doc id: 52e19838-8106-4152-bda4-82b49f3d0a92): The 1996-97 Utah Jazz season roster consisted of a group of players who represented the team duri...

> Source (Doc id: 17aebcbb-0093-4e06-b00b-1220d8eff22a): "1996-97 Utah Jazz season"...

> Source (Doc id: 76a6762f-4337-4a77-9b46-beb08a4080eb): "1996-97 Utah Jazz season"...


## Llama index multi doc case

In [65]:
import gradio as gr

In [66]:
# Environment, Loader, Query

def create_service_context():

    #constraint parameters
    max_input_size = 4096
    num_outputs = 512
    max_chunk_overlap = .5
    chunk_size_limit = 600

    #allows the user to explicitly set certain constraint parameters
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    #LLMPredictor is a wrapper class around LangChain's LLMChain that allows easy integration into LlamaIndex
    llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.5, model_name="gpt-3.5-turbo", max_tokens=num_outputs))
    #llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.5, model_name="gpt-4", max_tokens=num_outputs))

    #constructs service_context
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    return service_context

In [67]:
def data_ingestion_indexing(directory_path):

    #loads data from the specified directory path
    documents = SimpleDirectoryReader(directory_path).load_data()

    #when first building the index
    index = GPTVectorStoreIndex.from_documents(
        documents, service_context=create_service_context()
    )

    #persist index to disk, default "storage" folder
    index.storage_context.persist()

    return index

In [68]:
def data_querying(input_text):

    #rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="./storage")

    #loads index from storage
    index = load_index_from_storage(storage_context, service_context=create_service_context())

    #queries the index with the input text
    response = index.as_query_engine().query(input_text)

    return response.response

### Test Q&A across multiple documents using Gradio Web Interface

In [72]:
iface = gr.Interface(fn=data_querying,
                     inputs=gr.components.Textbox(lines=7, label="Enter your text"),
                     outputs="text",
                     title="🏀 Utah Jazz 🏀")

In [73]:
index = data_ingestion_indexing("/content")

In [74]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://159e8db93d6fb4b3cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## (optional) Cleanup

You may want to clean up your database: in that case, simply run the following cell.

_Warning: this will delete the vector store and all that you stored into it!_

In [ ]:
session.execute(f"DROP TABLE IF EXISTS {keyspace}.{table_name};")